<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter%C2%A010_%C2%A0Sequence_Hacking%2C_Hashing%2C_and_Slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Vector Take #1: Vector2d Compatible

In [0]:
from array import array
import reprlib
import math

In [0]:
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    #  we pass the memoryview directly to the constructor
    return cls(memv)

### Remark

The way I used reprlib.repr deserves some elaboration. That function produces safe
representations of large or recursive structures by limiting the length of the output string
and marking the cut with '...'. I wanted the repr of a Vector to look like Vector([3.0,
4.0, 5.0]) and not Vector(array('d', [3.0, 4.0, 5.0])), because the fact that
there is an array inside a Vector is an implementation detail. Because these constructor
calls build identical Vector objects, I prefer the simpler syntax using a list argument.


### Protocol
In the context of object-oriented programming, __a protocol is an informal interface,
defined only in documentation and not in code__. For example, the sequence protocol in
Python entails just the  --len-- and  --getitem-- methods. Any class Spam that imple‐
ments those methods with the standard signature and semantics can be used anywhere
a sequence is expected. Whether Spam is a subclass of this or that is irrelevant; all that

## Vector Take #2: A Sliceable Sequence

In [0]:
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  
  def __getitem__(self, index):
    return self._components[index]

In [7]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [9]:
v2 = Vector(range(10))

v2[1:4]

array('d', [1.0, 2.0, 3.0])

### Remark

As you can see, even slicing is supported—but not very well. It would be better if a slice
of a Vector was also a Vector instance and not a array. The old FrenchDeck class has
a similar problem: when you slice it, you get a list. In the case of Vector, a lot of
functionality is lost when slicing produces plain arrays.
Consider the built-in sequence types: every one of them, when sliced, produces a new
instance of its own type, and not of some other type.
To make Vector produce slices as Vector instances, we can’t just delegate the slicing to
array. We need to analyze the arguments we get in --getitem-- and do the right thing

In [0]:
import numbers
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    # Get the class of the instance (i.e., Vector) for later use.
    cls = type(self)
    
    # If the index argument is a slice…
    if isinstance(index, slice):
      
      # …invoke the class to build another Vector instance from a slice of the
      # _components array.
      return cls(self._components[index])
    
    # If the index is an int or some other kind of integer…
    elif isinstance(index, numbers.Integral):
      
      # …just return the specific item from _components. 
      return self._components[index]
    
    else:
      # Otherwise, raise an exception.
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))

In [16]:
v3 = Vector(range(7))
v3[-1]

6.0

In [17]:
v3[1:3]

Vector([1.0, 2.0])

In [18]:
v3[1, 4]

TypeError: ignored

## Vector Take #3: Dynamic Attribute Access